In [10]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import gc
from perturbench.analysis.preprocess import preprocess

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
data_directory = '../perturbench_data/'

In [ ]:
adata_paths = [
    'Seurat_object_IFNB_Perturb_seq.h5ad',
    'Seurat_object_IFNG_Perturb_seq.h5ad',
    'Seurat_object_INS_Perturb_seq.h5ad',
    'Seurat_object_TGFB_Perturb_seq.h5ad',
    'Seurat_object_TNFA_Perturb_seq.h5ad',
]
adata_paths = [data_directory + path for path in adata_paths]

In [ ]:
adata_list = []
for adata_path in adata_paths:
    adata = sc.read_h5ad(adata_path)
    adata.X = adata.raw.X.copy()
    adata.raw = None

    adata.obs.cell_type = [x.lower() for x in adata.obs.cell_type]
    adata.obs.cell_type.value_counts()

    adata.obs['treatment'] = adata_path.split('/')[-1].split('_')[2]
    adata.obs.treatment.value_counts()

    condition_remap = {
        'NT': 'control',
    }
    adata.obs['condition'] = adata.obs.gene.copy()
    adata.obs.condition = [condition_remap.get(x, x) for x in adata.obs.condition]
    adata.obs['condition'] = adata.obs.condition.astype('category')
    adata.obs['perturbation'] = adata.obs.condition.copy()

    adata.obs['ncounts'] = adata.obs['nCount_RNA'].copy()
    adata.obs['ngenes'] = adata.obs['nFeature_RNA'].copy()
    adata.obs['perturbation_type'] = 'CRISPRi'
    
    adata_list.append(adata)
    del adata
    gc.collect()

In [ ]:
adata_merged = ad.concat(adata_list)
adata_merged.obs_names_make_unique()

del adata_list
gc.collect()

adata_merged

In [ ]:
adata_merged.obs['dataset'] = 'jiang24'

In [ ]:
required_cols = [
    'condition',
    'cell_type',
    'treatment',
    'perturbation_type',
    'dataset',
    'ngenes',
    'ncounts',
]

for col in required_cols:
    assert col in adata_merged.obs.columns
    if np.any(adata_merged.obs[col].isnull()):
        print(col)
    if np.any(adata_merged.obs[col].isna()):
        print(col)

In [ ]:
adata_merged.obs.condition.value_counts()

condition
control    84269
IRF1       30300
MYC        24952
ELK1       23425
JUN        23145
           ...  
PPARG        844
SKP1         787
MCRS1        713
SMARCE1      689
SOX2         329
Name: count, Length: 219, dtype: int64

In [ ]:
adata_merged = preprocess(
    adata_merged,
    perturbation_key='condition',
    covariate_keys=['cell_type', 'treatment'],
)
adata_merged

Preprocessing ...
Filtering for highly variable genes or differentially expressed genes ...
Processed dataset summary:
View of AnnData object with n_obs × n_vars = 1628476 × 15476
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'bc1_well', 'bc2_well', 'bc3_well', 'percent.mito', 'cell_type', 'pathway', 'sample_ID', 'Batch_info', 'guide', 'gene', 'mixscale_score', 'treatment', 'condition', 'perturbation', 'ncounts', 'ngenes', 'perturbation_type', 'dataset', 'cov_merged', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'log1p', 'hvg', 'rank_genes_groups_cov'
   

View of AnnData object with n_obs × n_vars = 1628476 × 15476
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'bc1_well', 'bc2_well', 'bc3_well', 'percent.mito', 'cell_type', 'pathway', 'sample_ID', 'Batch_info', 'guide', 'gene', 'mixscale_score', 'treatment', 'condition', 'perturbation', 'ncounts', 'ngenes', 'perturbation_type', 'dataset', 'cov_merged', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'log1p', 'hvg', 'rank_genes_groups_cov'
    layers: 'counts'

In [ ]:
adata_merged.write_h5ad(data_directory + 'jiang24_processed.h5ad')